## Context

This notebook is an end-to-end example that you can follow to create a project, upload a first plugin and activate it on this project, and later see the plugin in action and monitor it.

**NB: The plugin capabilities of Kili are under active development, and compatible with version 2.125.2 & superior of kili. don't hesitate to reach out via github or the support to provide feedbacks.**


## Step 1 : Instantiate Kili with your API_KEY

In [ ]:
#%pip install kili

from kili.client import Kili
import os
import time

api_endpoint = os.getenv('KILI_API_ENDPOINT')
# If you use Kili SaaS, use the url 'https://cloud.kili-technology.com/api/label/v2/graphql'
api_endpoint = 'https://cloud.kili-technology.com/api/label/v2/graphql'

api_key = os.getenv('MY_KILI_API_KEY')

kili = Kili(api_endpoint=api_endpoint, api_key=api_key)

## Step 2 : Create the project

First, we need to create a new project. For our example, it will be a project of type `IMAGE` and it will have the following `jsonInterace` :

In [ ]:
json_interface = {
  "jobs": {
    "JOB_0": {
      "content": {
        "categories": {
          "OBJECT_A": {
            "children": [],
            "name": "Object A",
            "color": "#733AFB",
            "id": "category1"
          },
          "OBJECT_B": {
            "children": [],
            "name": "Object B",
            "color": "#3CD876",
            "id": "category2"
          }
        },
        "input": "radio"
      },
      "instruction": "Categories",
      "isChild": False,
      "tools": [
        "rectangle"
      ],
      "mlTask": "OBJECT_DETECTION",
      "models": {},
      "isVisible": True,
      "required": 1,
      "isNew": False
    }
  }
}

In [ ]:
# Create the project 

title = 'Plugins test project'
description = 'My first project with a plugin'
input_type = 'IMAGE'

project = kili.create_project(
    title=title,
    description=description,
    input_type=input_type,
    json_interface=json_interface
)
project_id = project["id"]

print(f'Created project {project_id}')

In [ ]:
# Upload an asset

content_array = ['./datasets/plugins/landscape.jpg']
names_array = ['landscape']

kili.append_many_to_dataset(
    project_id=project_id,
    content_array=content_array,
    external_id_array=names_array
)

asset_id = list(kili.assets(project_id=project_id, fields=["id"], disable_tqdm=True))[0]["id"]


This project has two jobs with several tasks of transcription associated. We will be interested in the *Payment information* job, and more precisely the *IBAN* and the *Currency* sub-jobs. With our plugin, we want to be sure that the labelers write the correct values for the two transcriptions (so we are interested only in the `on_submit` handler of the plugin), since we know for sure that the IBAN should start with the 2 letters *FR*, and that the currency should be one of : *EUR*, *USD*. At then end, we also calculate an accuracy and we insert it in the `json_metadata` of the asset.

To iterate on the plugin code, you can refer to *plugins_development.ipynb* notebook

## The plugin
The final plugin is in the `./datasets/plugins/plugin.py` file and its content is the following :

```python
from kili.plugins import PluginCore
from kili.types import Label

def check_rules_on_label(label: Label):
    #custom methods 
    print('Custom method - checking number of bboxes')

    counter = 0
    for annotation in label['jsonResponse']["JOB_0"]["annotations"]:
        if annotation["categories"][0]["name"] == "OBJECT_A":
            counter += 1

    if counter == 0:
        return []
    return [f'There are too many BBox ({counter}) - Only 1 BBox of Object A accepted']

class PluginHandler(PluginCore):
    """
    Custom plugin instance
    """

    def on_submit(self, label: Label, asset_id: str) -> None:
        """
        Dedicated handler for Submit action 
        """
        self.logger.info("On submit called")

        issues_array = check_rules_on_label(label)

        project_id = self.project_id

        if len(issues_array) > 0:
            print("Creating an issue...")

            for i, _ in enumerate(issues_array):

                self.kili.append_to_issues(
                    label_id=label['id'],
                    project_id=project_id,
                    text=issues_array[i],
                )

            print("Issue created!")

            self.kili.send_back_to_queue(asset_ids=[asset_id])

```

## Step 3 : Uploading the plugin

With the plugin defined in a separate `python` file, we can now upload it and activate it on our project.

- the upload will create the necessary builds to execute the plugin, it will take a few minutes.
- After the activation, you can start using it right away.

In [ ]:
path_to_plugin = './datasets/plugins/plugin.py'
plugin_name = 'Plugin bbox count'

kili.upload_plugin(path_to_plugin, plugin_name)

kili.activate_plugin_on_project(plugin_name, project_id=project_id)

## Step 4 : Seeing the plugin in action
You should wait for the plugin to be successfully deployed. After that, you can test it by labelling in the Kili interface or just by uploading the following label. You can see after that in the Kili app the issues created by the plugin (in case there are some errors).

In [ ]:
json_response = {
    "JOB_0": {
        "annotations": [
            {
                "boundingPoly": [
                    {
                        "normalizedVertices": [
                            {"x": 0.15, "y": 0.84},
                            {"x": 0.15, "y": 0.31},
                            {"x": 0.82, "y": 0.31},
                            {"x": 0.82, "y": 0.84}
                        ]
                    }
                ],
                "categories": [
                    {"name": "OBJECT_A"}
                ],
                "children": {},
                "mid": "20221124161451411-13314",
                "type": "rectangle"
            },
            {
                "boundingPoly": [
                    {
                        "normalizedVertices": [
                            {"x": 0.79, "y": 0.20},
                            {"x": 0.79, "y": 0.13},
                            {"x": 0.91, "y": 0.13},
                            {"x": 0.91, "y": 0.20}
                        ]
                    }
                ],
                "categories": [
                    {"name": "OBJECT_A"}
                ],
                "children": {},
                "mid": "20221124161456406-47055",
                "type": "rectangle"
            },
            {
                "boundingPoly": [
                    {
                        "normalizedVertices": [
                            {"x": 0.87, "y": 0.36},
                            {"x": 0.87, "y": 0.27},
                            {"x": 0.99, "y": 0.27},
                            {"x": 0.99, "y": 0.36}
                        ]
                    }
                ],
                "categories": [
                    {"name": "OBJECT_A"}
                ],
                "children": {},
                "mid": "20221124161459298-45160",
                "type": "rectangle"
            }
        ]
    }
}

In [ ]:
kili.append_labels(
   json_response_array=[json_response],
   asset_id_array=[asset_id],
   label_type='DEFAULT'
)

If you use the base plugin provided, the plugin should : 
 - create an issue with 1 comments precising that 3 bbox were encountered instead of one.
 - send back the asset back to the labeling queue (status `ONGOING`)

In [ ]:
# Let's check : 
print(kili.assets(project_id=project_id, asset_id=asset_id, fields=['status', 'issues.comments.text']))

# You can also access the app at the link 
print(f'Go to my project {api_endpoint.split("/api")[0]}/label/projects/{project_id}/menu/queue')

Woah amazing !! Well done :) 🚀
Let's test now to post a proper label, this one for example : 

In [ ]:
json_response = {
    "JOB_0": {
        "annotations": [
            {
                "boundingPoly": [
                    {
                        "normalizedVertices": [
                            {"x": 0.15, "y": 0.84},
                            {"x": 0.15, "y": 0.31},
                            {"x": 0.82, "y": 0.31},
                            {"x": 0.82, "y": 0.84}
                        ]
                    }
                ],
                "categories": [
                    {"name": "OBJECT_A"}
                ],
                "children": {},
                "mid": "20221124161451411-13314",
                "type": "rectangle"
            }
        ]
    }
}
kili.append_labels(
   json_response_array=[json_response],
   asset_id_array=[asset_id],
   label_type='DEFAULT'
)

time.sleep(10)
# Let's check : 
print(kili.assets(project_id=project_id, asset_id=asset_id, fields=['status']))

# You can also access the app at the link 
print(f'Go to my project {api_endpoint.split("/api")[0]}/label/projects/{project_id}/menu/queue')

Now you should see the status `LABELED` for your asset. In this plugin, former issues remain but could be solved through the api as well ! 


Well done, you can iterate on the script now, for this, to avoid the latency necessary to build & deploy your plugin, don't hesitate to use [this notebook](./plugins_development.ipynb)

## Step 5 : Monitoring the plugin
In order to monitor a certain plugin, you can get its logs by using the following command :

In [ ]:
import json
from datetime import date
from datetime import datetime
dt = date.today()  # You can change this date if needed, or omit it to set it at the plugin creation date
start_date = datetime.combine(dt, datetime.min.time())

logs = kili.get_plugin_logs(project_id=project_id, plugin_name=plugin_name, start_date=start_date)

logs_json = json.loads(logs)
print(json.dumps(logs_json, indent=4))

## Step 6 : Managing the plugin
You can also use the following methods to better manage your plugins

In [ ]:
# Get the list of all uploaded plugins in your organization
plugins = kili.list_plugins()

In [ ]:
# Update a plugin with new source code
kili.update_plugin(plugin_name=plugin_name, file_path=path_to_plugin)

In [ ]:
# Deactivate the plugin on a certain project (the plugin can still be active for other projects)
kili.deactivate_plugin_on_project(plugin_name=plugin_name, project_id=project_id)

In [ ]:
# Delete the plugin completely (deactivates automatically the plugin from all projects)
kili.delete_plugin(plugin_name=plugin_name)